In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Linear-Regression').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(featuresCol='features',
                      labelCol='label',
                     predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
training_summary = lrModel.summary

In [15]:
training_summary.r2

0.027839179518600154

In [16]:
training_summary.rootMeanSquaredError

10.16309157133015

In [18]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [19]:
split_object = all_data.randomSplit([0.7,0.3])

In [20]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [21]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 343|
|   mean|0.016490865035544866|
| stddev|  10.057036419977074|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [27]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                158|
|   mean| 0.7787654000791063|
| stddev| 10.877215493829405|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [28]:
correct_model = lr.fit(train_data)

In [29]:
test_results = correct_model.evaluate(test_data)

In [30]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -27.60484405948727|
| -21.20901866381476|
|-23.287876443399572|
|-22.824828326676272|
|-26.809853386163596|
|-21.823110175074063|
| -21.38612222374523|
|-19.325991680445817|
|-18.490231572075437|
|-13.852430448931695|
| -19.28349139505542|
|-20.041585577219934|
|-14.306426169723778|
|-13.158596991931143|
|-16.260226150779005|
|-14.455695332736173|
|-11.122091273959715|
|-11.604982279289056|
|-12.939045491141663|
| -13.30005760589053|
+-------------------+
only showing top 20 rows



In [31]:
test_results.rootMeanSquaredError

11.0115501763839

In [32]:
unlabeled_data = test_data.select('features')

In [33]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [34]:
predictions = correct_model.transform(unlabeled_data) 

In [35]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.44117397828936383|
|(10,[0,1,2,3,4,5,...| -5.5271885187869625|
|(10,[0,1,2,3,4,5,...|-0.22300764692339753|
|(10,[0,1,2,3,4,5,...| -0.6626117942602385|
|(10,[0,1,2,3,4,5,...|   3.860027449967523|
|(10,[0,1,2,3,4,5,...|  0.3907224109082553|
|(10,[0,1,2,3,4,5,...|  1.3286396079560159|
|(10,[0,1,2,3,4,5,...| -0.5585690938276087|
|(10,[0,1,2,3,4,5,...|  -1.382759465992969|
|(10,[0,1,2,3,4,5,...|  -5.315862174032066|
|(10,[0,1,2,3,4,5,...|  1.0082778290507888|
|(10,[0,1,2,3,4,5,...|  2.6129110062804255|
|(10,[0,1,2,3,4,5,...|  -1.069431553588518|
|(10,[0,1,2,3,4,5,...| -1.6041612609999842|
|(10,[0,1,2,3,4,5,...|   1.931247641703564|
|(10,[0,1,2,3,4,5,...|  0.4795644015834706|
|(10,[0,1,2,3,4,5,...|  -2.298503501931042|
|(10,[0,1,2,3,4,5,...| -1.3728664461030473|
|(10,[0,1,2,3,4,5,...|  0.7409489264802509|
|(10,[0,1,2,3,4,5,...|  1.421890

In [ ]:
## Mean Absolute Error
### this is the Mean of the absolute value of errors
### easy to understand just an average error
